# 빌딩 검출

## 파라미터 : 
*  epoch : 30
* beta1: 0.9, beta2 : 0.999 
* learning rate : 0.0001

## 라벨링 방식 : 흑백 채우기 

In [1]:
!pip install segmentation_models

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import math
import os
import glob
import json
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import re
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import tensorflow as tf

from tensorflow import keras 
import numpy as np
import matplotlib.pyplot as plt

In [4]:
!ls
!ls /content/drive/MyDrive/SIA/colab_code/lhk/

'=0.3.0'
 0603_roads_label01_v1.h5
'[AIFFEL해커톤][SIA] 데이터 전달, 멘토링 방법_202105.pptx'
 best_model.h5
 best_model.pth
 binary_roads_imagepatch.ipynb
'BnR 라벨링 전처리.ipynb'
 BnR_train_list.txt
 BnR_val_list.txt
'Building Detection'
 buildings_eda.ipynb
'cars segmentation (camvid).ipynb'
 data
 multi_bnr_augmentation.ipynb
'multiclass segmentation (camvid)_원본.ipynb'
'parsing BnR .ipynb'
 parsing_building.ipynb
 parsing_roads.ipynb
 투명.png
 preprocessing
'Road Detection'
 road_segmentation_v1.ipynb
 road_segmentation_v3.ipynb
'Segmentation Models - building.ipynb'
'Segmentation Models- pytorch.ipynb'
 SegNet
'SIA 해커톤 과업지시서.pdf'
 UnetBuildingtrain.ipynb
ls: cannot access '/content/drive/MyDrive/SIA/colab_code/lhk/': No such file or directory


In [5]:
DATA_DIR =  './data/buildings/train/'
VAL_DIR = './data/buildings/val/'

In [42]:
B_train_image_path = './data/buildings/train/raw_image/' #빌딩 트레인 input image 경로
B_train_label_path = './data/buildings/train/new_label/' #빌딩 트레인 변환된 label 경로

B_val_image_path = './data/buildings/val/raw_image/' #빌딩 validation input image 경로 
B_val_label_path = './data/buildings/val/new_label/' #빌딩 validation 변환된 label image 경로

R_train_image_path = './data/roads/train/raw_image/' #도로 트레인 input image 경로 
R_train_label_path = './data/roads/train/new_label/' #도로 트레인 변환된 label 경로

R_val_image_path = './data/roads/val/raw_image/' #도로 validation input image 경로
R_val_label_path = './data/roads/val/new_label/' #도로 트레인 변환된 label 경로 

In [43]:
BnR_train_png_files = glob.glob(B_train_image_path + '*.png')
B_train_label_png_files = glob.glob(B_train_label_path + '*.png')
R_train_label_png_files = glob.glob(R_train_label_path + '*.png')

BnR_val_png_files = glob.glob(B_val_image_path + '*.png')
B_val_label_png_files = glob.glob(B_val_label_path + '*.png')
R_val_label_png_files = glob.glob(R_val_label_path + '*.png')

R_train_png_files =  glob.glob(R_train_image_path + '*.png')
R_val_png_files =   glob.glob(R_val_image_path + '*.png')

In [44]:
## lv2 데이터셋의 train과 validation은  기존(lv1)의 train, validation에서 혼합되어 나옴 

## 그러므로 기존 train, validation 명단을 합쳐서 명단을 만들어냄 
BnR_input_png_files = BnR_train_png_files + BnR_val_png_files # 빌딩은 BnR으로 그대로 쓰면됨 
R_input_png_files = R_train_png_files + R_val_png_files

B_label_png_files = B_train_label_png_files + B_val_label_png_files
R_label_png_files = R_train_label_png_files + R_val_label_png_files

# 전체 파일 목록

In [45]:
## 위에서 만든 전체 파일은 train, validation 파일이기때문에 파일의 경로들이 다 다름.
## 이를 해결하기 위해서 파일명만 남기게 함수를 만들어줌 

def file_name_only(files_list):
    filename_only = []
    for i in files_list:
        filename_only.append(i.split('/')[-1])
    return filename_only

In [46]:
BnR_input_png_files_only = file_name_only(BnR_input_png_files)
B_label_png_files_only =file_name_only(B_label_png_files)
R_label_png_files_only = file_name_only(R_label_png_files)

## Lv2 에 쓸 파일 목록명단이랑 체크하기

In [47]:
def check_lv2_list(lv2_list,png_filesname_only,file_path_included):
    lv2 = []
    
    f = open(lv2_list,'r')
    lines = f.readlines()
    for line in lines:
        a = line.split('.')[0] + '.png'
        if a in png_filesname_only:
            ind = png_filesname_only.index(a)
            lv2.append(file_path_included[ind])
    f.close()
    
    return lv2

## Lv2 train

In [48]:
BnR_train_input_list = check_lv2_list("./BnR_train_list.txt",BnR_input_png_files_only,BnR_input_png_files)
Bnr_train_label_list = check_lv2_list("./BnR_train_list.txt",B_label_png_files_only,B_label_png_files)
bnR_train_label_list = check_lv2_list("./BnR_train_list.txt",R_label_png_files_only,R_label_png_files)

## lv2 val

In [49]:
BnR_val_input_list = check_lv2_list("./BnR_val_list.txt",BnR_input_png_files_only,BnR_input_png_files)
Bnr_val_label_list = check_lv2_list("./BnR_val_list.txt",B_label_png_files_only,B_label_png_files)
bnR_val_label_list = check_lv2_list("./BnR_val_list.txt",R_label_png_files_only,R_label_png_files)

# 명단에 들어가는 파일만 체크

In [67]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    

# classes for data loading and preprocessing
class Dataset_multi:
    CLASSES =  ['roads','buildings']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir_b, 
            masks_dir_r,
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = [i.split('/')[-1] for i in images_dir]
        self.images_fps = images_dir
        self.masks_fps_b = masks_dir_b
        self.masks_fps_r = masks_dir_r
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask_b = cv2.imread(self.masks_fps_b[i], 0)
        mask_r = cv2.imread(self.masks_fps_r[i], 0)
        mask_r = np.where(mask_b != mask_r, mask_r, 0)
        # extract certain classes from mask (e.g. cars)
        mask = np.stack((mask_b,mask_r), axis=-1).astype('float')
        
        # extract certain classes from mask (e.g. cars)
#         masks = [(mask == v) for v in self.class_values]
#         mask = np.stack(masks, axis=-1).astype('float')
        
        # add background if mask is not binary
        if mask.shape[-1] != 1:
            # 둘다 1일 경우..
            background = 255 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
            mask = np.where(mask == 255, mask, 0)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
#         image = image/255         
        mask = mask/255
        return image, mask
        
    def __len__(self):
        return len(self.ids)

In [68]:
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
       
        return tuple(batch)
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)   

In [69]:

from albumentations import (
                            Compose,
                            HorizontalFlip,
                            ShiftScaleRotate,
                            OneOf,
                            RandomContrast,
                            RandomGamma,
                            RandomBrightness
   
                            )


def augmentations(prob=0.5):
    
    transformer = Compose([
                      
            HorizontalFlip(p=prob),
            ShiftScaleRotate(p=prob, shift_limit=0.1, scale_limit=.1, rotate_limit=10),
            OneOf([RandomContrast(limit=0.1, p=prob),
                   RandomGamma(gamma_limit=(90, 110), p=prob),
                   RandomBrightness(limit=0.1, p=prob)],p=prob),
            
    ], p=prob)
    return transformer

In [70]:
# def get_validation_augmentation():
#     """Add paddings to make image shape divisible by 32"""
#     test_transform = [
#         A.PadIfNeeded(1024, 1024)
#     ]
#     return A.Compose(test_transform)


In [71]:
# def get_preprocessing(preprocessing_fn):
#     """Construct preprocessing transform
    
#     Args:
#         preprocessing_fn (callbale): data normalization function 
#             (can be specific for each pretrained neural network)
#     Return:
#         transform: albumentations.Compose
    
#     """
    
#     _transform = [
#         A.Lambda(image=preprocessing_fn),
#     ]
#     return A.Compose(_transform)

In [72]:
%env SM_FRAMEWORK=tf.keras

import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras


In [73]:
CLASSES =  ['roads','buildings']

In [74]:
train_dataset = Dataset_multi(
    BnR_train_input_list, 
    Bnr_train_label_list, bnR_train_label_list,
    classes=CLASSES,
    augmentation=augmentations(),
    #preprocessing=get_preprocessing(preprocess_input),
)

In [75]:
len(bnR_train_label_list)

324

In [76]:
valid_dataset = Dataset_multi(
    BnR_val_input_list, 
    Bnr_val_label_list, bnR_val_label_list, 
    classes=CLASSES,    
    augmentation=augmentations,
   # preprocessing=get_preprocessing(preprocess_input)

)


## 모델 설계

In [77]:
BACKBONE = 'efficientnetb3'
BATCH_SIZE = 1
LR = 0.0001
EPOCHS = 30
preprocess_input = sm.get_preprocessing(BACKBONE)

In [78]:
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

In [79]:
# define optomizer
optim = tf.keras.optimizers.Adam(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
dice_loss = sm.losses.DiceLoss(class_weights=np.array([1, 2, 0.5])) 
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [80]:
train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)

In [81]:
# assert train_dataloader[0][0].shape == (BATCH_SIZE, 1024, 1024, 3)
# assert train_dataloader[0][1].shape == (BATCH_SIZE, 1024, 1024, n_classes)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint('aug_test.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
]

In [82]:
model.summary

<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7f147c4c70d0>>

In [84]:
# train model
history = model.fit(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader), 
    epochs=30, 
     callbacks=callbacks,      validation_data=valid_dataloader, 
    validation_steps=len(valid_dataloader) ,verbose = 1 
)

Epoch 1/30
115/324 [=========>....................] - ETA: 22:41 - loss: 0.5504 - iou_score: 0.3922 - f1-score: 0.5217

KeyboardInterrupt: 

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 8))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## 모델 평가

In [ ]:
test_dataset = Dataset_multi(
    BnR_val_input_list, 
    Bnr_val_label_list, bnR_val_label_list, 
    classes=CLASSES,    
#     augmentation=get_validation_augmentation(),
#         preprocessing=get_preprocessing(preprocess_input)

)
test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

In [ ]:

# load best weights
model.load_weights('aug_test.h5')


In [ ]:
scores = model.evaluate_generator(test_dataloader)

print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric.__name__, value))

In [ ]:
n = 5
ids = np.random.choice(np.arange(len(test_dataset)), size=n)

for i in ids:
    
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image)
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask.squeeze(),
        pr_mask=pr_mask.squeeze(),
    )

In [ ]:
n = 5
ids = np.random.choice(np.arange(len(test_dataset)), size=n)

for i in ids:
    
    image, gt_mask = test_dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image)
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask.squeeze(),
        pr_mask=pr_mask.squeeze(),
    )